<a href="https://colab.research.google.com/github/tandav/gists/blob/master/deepspeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install deepspeech-gpu==0.5.0a4
!git clone https://github.com/mozilla/DeepSpeech /DeepSpeech
    
import os
os.chdir('/DeepSpeech')
!pip3 install -r requirements.txt

!pip3 install $(python3 util/taskcluster.py --decoder)
!git clone https://github.com/GeorgeFedoseev/DeepSpeech /GeorgeFedoseevDeepSpeech
    
!mv /DeepSpeech/data /DeepSpeech/moz_data
!mv /GeorgeFedoseevDeepSpeech/data /DeepSpeech/data

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def download_gdrive_file(file_id):
    '''
    example file_id:
    1FdiUVpef4cVKnB8noG4BUjVTSwaS9HfA
    '''
    gfile = drive.CreateFile({'id': file_id})
    gfile.FetchMetadata()
    gfile_name = gfile['title']
    gfile.GetContentFile(gfile_name)

In [0]:
download_gdrive_file('1FdiUVpef4cVKnB8noG4BUjVTSwaS9HfA') # LM-mixed-yt-echo-wiki-o5-prune2-24Jun18 (1GB)
!tar xvfz LM-mixed-yt-echo-wiki-o5-prune2-24Jun18.tar.gz

mixed-yt-echo-wiki-o5-prune2-24Jun18/
mixed-yt-echo-wiki-o5-prune2-24Jun18/vocabulary.txt
mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary
mixed-yt-echo-wiki-o5-prune2-24Jun18/trie
mixed-yt-echo-wiki-o5-prune2-24Jun18/words.arpa
mixed-yt-echo-wiki-o5-prune2-24Jun18/alphabet.txt


In [0]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
#                     print('.')

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    



file_id = '12WUh8REDuhOAQIISF7hBldM0mhxtDpE0'
destination = 'yt-vad-650-clean.tar.gz'
download_file_from_google_drive(file_id, destination)

!tar xfz yt-vad-650-clean.tar.gz

tar: Removing leading `/' from member names


In [0]:
!mv /DeepSpeech/headless/shared/Deep-Speech-Assets/DATASETS /datasets

In [0]:
(0.1 + 0.35)/2

0.22499999999999998

In [0]:
import pandas as pd

# frac = 0.1  # ok 
# frac = 0.35 # killed
# frac = 0.5  # killed
# frac = 0.22

frac = 0.001

pd.read_csv('/datasets/yt-vad-650-clean/yt-vad-650-clean-train.csv').sample(frac=frac).to_csv('/datasets/yt-vad-650-clean/subsample-train.csv', index=False)
pd.read_csv('/datasets/yt-vad-650-clean/yt-vad-650-clean-dev.csv').sample(frac=frac).to_csv('/datasets/yt-vad-650-clean/subsample-dev.csv', index=False)
pd.read_csv('/datasets/yt-vad-650-clean/yt-vad-650-clean-test.csv').sample(frac=frac).to_csv('/datasets/yt-vad-650-clean/subsample-test.csv', index=False)
!wc -l /datasets/yt-vad-650-clean/subsample-train.csv
!wc -l /datasets/yt-vad-650-clean/subsample-dev.csv
!wc -l /datasets/yt-vad-650-clean/subsample-test.csv

302 /datasets/yt-vad-650-clean/subsample-train.csv
29 /datasets/yt-vad-650-clean/subsample-dev.csv
29 /datasets/yt-vad-650-clean/subsample-test.csv


In [0]:
# !python DeepSpeech.py --helpfull

In [0]:
#   --train_files "/DeepSpeech/data/tiny-dataset/tiny-train.csv" \
#   --dev_files "/DeepSpeech/data/tiny-dataset/tiny-dev.csv" \
#   --test_files "/DeepSpeech/data/tiny-dataset/tiny-test.csv" \

  
#   --lm_trie_path '/DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/trie' \
ex = '''
#!/bin/sh
set -xe
if [ ! -f DeepSpeech.py ]; then
    echo "Please make sure you run this from DeepSpeech's top level directory."
    exit 1
fi;


# kinda wrecked it (made all numbers small af)

python -u DeepSpeech.py \
  --train_files "/datasets/yt-vad-650-clean/subsample-train.csv" \
  --dev_files "/datasets/yt-vad-650-clean/subsample-dev.csv" \
  --test_files "/datasets/yt-vad-650-clean/subsample-test.csv" \
  --train_batch_size 1 \ob
  --dev_batch_size 1 \
  --test_batch_size 1 \
  --learning_rate 0.0001 \
  --epoch 1 \
  --display_step 0 \
  --validation_step 1 \
  --dropout_rate 0.3 \
  --default_stddev 0.046875 \
  --n_hidden 512 \
  --checkpoint_dir /deep_checkpoints/ \
  --export_dir /deep_export/ \
  --log_level 2 \
  --report_count 3 \
  --xla=False \
  --use_warpctc=True \
  --lm_binary_path '/DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary' \
'''

with open('my_train.sh', 'w') as f:
    f.write(ex)
!bash my_train.sh

+ '[' '!' -f DeepSpeech.py ']'
+ python -u DeepSpeech.py --train_files /datasets/yt-vad-650-clean/subsample-train.csv --dev_files /datasets/yt-vad-650-clean/subsample-dev.csv --test_files /datasets/yt-vad-650-clean/subsample-test.csv --train_batch_size 1 --dev_batch_size 1 --test_batch_size 1 --learning_rate 0.0001 --epoch 1 --display_step 0 --validation_step 1 --dropout_rate 0.3 --default_stddev 0.046875 --n_hidden 512 --checkpoint_dir /deep_checkpoints/ --export_dir /deep_export/ --log_level 2 --report_count 3 --xla=False --use_warpctc=True --lm_binary_path /DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary
Instructions for updating:
Colocations handled automatically by placer.
Preprocessing ['/datasets/yt-vad-650-clean/subsample-train.csv']
Preprocessing done
Preprocessing ['/datasets/yt-vad-650-clean/subsample-dev.csv']
Preprocessing done
Instructions for updating:
Use tf.cast instead.
100% (301 of 301) |#######################| Elapsed Time: 0:00:41 Time:  0:00:41
100% (28

In [72]:
import datetime
timestamp = str(datetime.datetime.now()).replace(' ', '__')
zipname = f'deep_checkpoints__{timestamp}.zip'

!zip -r {zipname} /deep_checkpoints

gdrive_file_to_upload = zipname

# Create & upload a file.
uploaded = drive.CreateFile({'title': gdrive_file_to_upload})
uploaded.SetContentFile(gdrive_file_to_upload)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

  adding: deep_checkpoints/ (stored 0%)
  adding: deep_checkpoints/model.ckpt-301.meta (deflated 89%)
  adding: deep_checkpoints/events.out.tfevents.1554119478.ba9886341b3e (deflated 88%)
  adding: deep_checkpoints/checkpoint (deflated 65%)
  adding: deep_checkpoints/model.ckpt-0.meta (deflated 89%)
  adding: deep_checkpoints/graph.pbtxt (deflated 96%)
  adding: deep_checkpoints/model.ckpt-0.index (deflated 56%)
  adding: deep_checkpoints/model.ckpt-301.index (deflated 47%)
  adding: deep_checkpoints/model.ckpt-0.data-00000-of-00001 (deflated 69%)
  adding: deep_checkpoints/model.ckpt-301.data-00000-of-00001 (deflated 7%)
Uploaded file with ID 1Wo8stxLTw0j_fmv37PBznnf3VI9RQThG


In [0]:
!deepspeech --model /deep_export/output_graph.pb --alphabet /DeepSpeech/data/alphabet.txt --lm /DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary --trie /DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/trie --audio /DeepSpeech/data/tiny-dataset/tiny-test/ru_0054.wav

Loading model from file /deep_export/output_graph.pb
TensorFlow: v1.13.1-8-g17f8188
DeepSpeech: v0.5.0-alpha.4-0-gc092213
2019-03-29 14:04:09.202552: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-03-29 14:04:09.296570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-29 14:04:09.297035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2019-03-29 14:04:09.297075: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible gpu devices: 0
2019-03-29 14:04:09.591846: I tensorflow/core/common_runtime/gpu/gpu_device.cc:984] Device interconnect StreamExecutor wit

In [0]:
# !tar xvfz samples-from-6k-dataset-28Jun18.tar.gz

!tar xfvz yt-vad-650-clean.tar.gz

mixed-yt-echo-wiki-o5-prune2-24Jun18/
mixed-yt-echo-wiki-o5-prune2-24Jun18/vocabulary.txt
mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary
mixed-yt-echo-wiki-o5-prune2-24Jun18/trie
mixed-yt-echo-wiki-o5-prune2-24Jun18/words.arpa
mixed-yt-echo-wiki-o5-prune2-24Jun18/alphabet.txt
tar: Removing leading `/' from member names



In [0]:
# !ps aux | grep -v grep | grep DeepSpeech.py
# !kill 2411

root        2411  182 79.2 51091456 10572332 ?   Sl   15:42  58:16 python3 -u DeepSpeech.py --train_files /datasets/yt-vad-650-clean/subsample-train.csv --dev_files /datasets/yt-vad-650-clean/subsample-dev.csv --test_files /datasets/yt-vad-650-clean/subsample-test.csv --train_batch_size 1 --dev_batch_size 1 --test_batch_size 1 --learning_rate 0.0001 --epoch 1 --display_step 0 --validation_step 1 --dropout_rate 0.3 --default_stddev 0.046875 --n_hidden 10 --checkpoint_dir /deep_checkpoints/ --export_dir /deep_export/ --log_level 2 --report_count 3 --xla=False --use_warpctc=True --lm_binary_path /DeepSpeech/mixed-yt-echo-wiki-o5-prune2-24Jun18/lm.binary


In [0]:
!nvidia-smi

Sat Mar 30 13:44:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!bash bin/train-tiny-dataset.sh

In [0]:
!python /DeepSpeech/DeepSpeech.py --train_files /DeepSpeech/data/tiny-dataset/tiny-train.csv --dev_files /DeepSpeech/data/tiny-dataset/tiny-dev.csv --test_files /DeepSpeech/data/tiny-dataset/tiny-test.csv

In [0]:
# !python /DeepSpeech/DeepSpeech.py --train_files /DeepSpeech/data/tiny-dataset/tiny-train.csv --dev_files /DeepSpeech/data/tiny-dataset/tiny-dev.csv --test_files /DeepSpeech/data/tiny-dataset/tiny-test.csv

In [0]:
# this link changes every day
# https://drive.google.com/uc?id=12WUh8REDuhOAQIISF7hBldM0mhxtDpE0&export=download
!curl 'https://doc-08-a4-docs.googleusercontent.com/docs/securesc/k2cjvm9vil57g6ltq771ukbo5ulegnap/k6tlcgjgum46kfanmka6tgfmr0gt7bj7/1553947200000/08032243308062674117/02123041894493518740/12WUh8REDuhOAQIISF7hBldM0mhxtDpE0?e=download&nonce=08h0fk4l23ptu&user=02123041894493518740&hash=2p1bg486trla047l8c6tbaa0kajitajq' -H 'authority: doc-08-a4-docs.googleusercontent.com' -H 'upgrade-insecure-requests: 1' -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3' -H 'x-client-data: CIW2yQEIprbJAQjBtskBCKmdygEIqKPKAQixp8oBCOKoygEI8KnKAQiurMoB' -H 'referer: https://drive.google.com/uc?export=download&confirm=mw0k&id=12WUh8REDuhOAQIISF7hBldM0mhxtDpE0' -H 'accept-encoding: gzip, deflate, br' -H 'accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' -H 'cookie: AUTH_9cv7ish6b224rq3kvehhahj324m7enso_nonce=08h0fk4l23ptu; NID=152=ffTx9sKwR1XhKRI0zz0-B7AB4tcJdfqItemY2ln2Kag4MNiNxJEmFbDSG8YJ0W9VpzFTPngnerrLqNpHqpon8NWzFuJQpu7JdV1Q6ppGvo0nkAcByeBRzD53-dosfSK8fDsp3nsjO6g3ESJh8O_omJPvjQRx9SfrvHWT1EyoKEo' --compressed > 'yt-vad-650-clean.tar.gz'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
